<a href="https://colab.research.google.com/github/Madumali-Prasad/Madumali-Prasad/blob/main/Get_tweet_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy 
from tqdm import tqdm
import re
import datetime
import pandas as pd

#Twitter API credentials
consumer_key = "vKGMwbp2xKlxaM1kvyWMUTyxx"
consumer_secret = "UAAuvczizcqGqD6A7NcMaM4eemPVumqadT7ZRJsfBS5pEVJiJn"
access_key = "954661768915275776-m8w4eUgc0TLth6h5i9S8iswIiVSFwHx"
access_secret = "lsdKaFayiH1Auik0oi2fOyKusVeiR8uIFW6iecKAOk9Cu"

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import datetime 

try:
 auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
 auth.set_access_token(access_key, access_secret)
 auth.get_authorization_url()
 api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True,compression=True,retry_count=3,retry_delay=10,timeout=15)
except tweepy.TweepError:
 print ('Error')

name="OxfordEconomics"

startDate = datetime.datetime(2020,8, 15, 5, 00, 00)
endDate =   datetime.datetime(2022, 5, 31, 13, 00, 00)

say=0
tweets = []
from time import sleep
tmpTweets = api.user_timeline(name,count=200,tweet_mode="extended",lang="tr")

for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)

lastTweet = tmpTweets[-1].id
while (tmpTweets[-1].created_at > startDate):
    #print("SriLankaTweet @", tmpTweets[-1].created_at,say)

    tmpTweets = api.user_timeline(name,max_id = tmpTweets[-1].id,tweet_mode="extended")
    if lastTweet == tmpTweets[-1].id:
        print("lastTweet")
        sleep(2)
    else:
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                tweets.append(tweet)
    lastTweet = tmpTweets[-1].id
    say+=1

print("Done Extraction")

T_Text=[]
T_Create=[]
T_id=[]
for i in tweets:
  T_id.append(i.id)
  T_Create.append(str(i.created_at))
  T_Text.append(i.full_text)

data_tweets=pd.DataFrame(list(zip(T_id,T_Create,T_Text)),columns=["id","created_at","text"])
print(f"There are {len(data_tweets)} tweets filtered in finance sector")

Done Extraction
There are 2615 tweets filtered in finance sector


## Getting Predictions

In [ ]:
#Getting Predictions
#Libraries
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
import joblib
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
vectorizing_model = joblib.load("/content/Tweet_Vectorizer ") 
GS_model=joblib.load("/content/Best_model.sav") 

In [ ]:
##Function to clean tweets

stopwords_list = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()
punctuations='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
lst=list(punctuations)

def remove_unwanted(review):
    splitReview = review.split()
    splitReview = [word for word in splitReview if not word in stopwords_list] #Remove unwanted words
    splitReview = [lemmatizer.lemmatize(word) for word in splitReview]  #Change the tense of the word
    splitReview = [porter_stemmer.stem(word) for word in splitReview]  #Remove stemming
    clean_review = ' '.join(splitReview)
    return clean_review

def clean_text(text):
    text=text.lower()
    for i in lst:
        text = text.replace(i,'')
    clean_text= remove_unwanted(text)
    return clean_text

In [ ]:
All_prds=[]
pred_prob=[]
predict_labels={0:"Negative Review",1:"Neutral Review",2:"Positive Review"}
for idx in range(len(data_tweets)):
  cont=data_tweets.iloc[idx,2]
  comment=cont.split('https')[0].strip()
  comment_clean=clean_text(comment)
  vectorized_comment=vectorizing_model.transform([comment_clean])
  prd=GS_model.predict(vectorized_comment)

  probability=GS_model.predict_proba(vectorized_comment)
  pred_prob.append(probability[0][2])
  All_prds.append(predict_labels[prd[0]])
  #print(f"Tweet:{comment}")
  #print(f"Prediction: {predict_labels[prd[0]]}\n")

print(len(data_tweets),len(All_prds),len(pred_prob))

2615 2615 2615


In [ ]:
#Final Prediction Dataframe
data_tweets["Sentiment"]=All_prds
data_tweets["Positivity"]=pred_prob
data_tweets.head(10)

,id,created_at,text,Sentiment,Positivity
0,1531214374974660608,2022-05-30 10:02:10,Now the big concern about inflation is where i...,Positive Review,0.398952
1,1531180678255083521,2022-05-30 07:48:16,APACs short-term economic outlook has deterior...,Neutral Review,0.297345
2,1531083184259092480,2022-05-30 01:20:51,Register here for our webinar presentation on ...,Negative Review,0.314178
3,1530131888127545344,2022-05-27 10:20:45,"The scale of the increases in debt costs, coup...",Negative Review,0.216324
4,1530104973824086016,2022-05-27 08:33:48,哪些行业及领域能够成为稳增长的主要抓手？稳经济发力关键点为何？在近期接受的访问中，牛津经济研...,Negative Review,0.343645
5,1529851356483399681,2022-05-26 15:46:01,There is much speculation about Mexico being o...,Negative Review,0.360020
6,1529828881582149635,2022-05-26 14:16:42,Join Oxford Economics and Experian for the Qua...,Negative Review,0.363155
7,1529828858261819393,2022-05-26 14:16:37,Join Oxford Economics and Experian for the Qua...,Negative Review,0.363155
8,1529824932871274496,2022-05-26 14:01:01,A 10% correction is needed for US gateway indu...,Negative Review,0.213027
9,1529809830910873601,2022-05-26 13:01:00,"Risks of a US recession have increased, partic...",Positive Review,0.377584


In [ ]:
from datetime import datetime
d=[]
for _,row in data_tweets.iterrows():
  d.append(datetime.strptime(row["created_at"], '%Y-%m-%d %H:%M:%S').date())

In [ ]:
#Add date colummn
df_new=data_tweets.copy()
df_new["date_col"]=d

df_new.head(3)

,id,created_at,text,Sentiment,Positivity,date_col
0,1531214374974660608,2022-05-30 10:02:10,Now the big concern about inflation is where i...,Positive Review,0.398952,2022-05-30
1,1531180678255083521,2022-05-30 07:48:16,APACs short-term economic outlook has deterior...,Neutral Review,0.297345,2022-05-30
2,1531083184259092480,2022-05-30 01:20:51,Register here for our webinar presentation on ...,Negative Review,0.314178,2022-05-30


In [ ]:
df_final=df_new.groupby('date_col')['Positivity'].mean()
idx = pd.date_range('08-15-2020', '05-31-2022')
df_final.index = pd.DatetimeIndex(df_final.index)
df_final = df_final.reindex(idx, fill_value=0)
print(df_final)

2020-08-15    0.000000
2020-08-16    0.000000
2020-08-17    0.243951
2020-08-18    0.369388
2020-08-19    0.366992
                ...   
2022-05-27    0.279985
2022-05-28    0.000000
2022-05-29    0.000000
2022-05-30    0.336825
2022-05-31    0.000000
Freq: D, Name: Positivity, Length: 655, dtype: float64


In [ ]:
import numpy as np
for i in range(len(df_final)):
  if df_final[i]==0:
    df_final[i]=df_final.mean()
print(df_final)

2020-08-15    0.254663
2020-08-16    0.255051
2020-08-17    0.243951
2020-08-18    0.369388
2020-08-19    0.366992
                ...   
2022-05-27    0.279985
2022-05-28    0.341850
2022-05-29    0.342372
2022-05-30    0.336825
2022-05-31    0.342894
Freq: D, Name: Positivity, Length: 655, dtype: float64


In [ ]:
df_final.to_csv("data_tweets.csv")